<a href="https://colab.research.google.com/github/Dirkster99/ChessMatingPatterns/blob/main/CoLab_Python_Parser_V2/02_Filter_lichess_db_for_Vukovic_mate_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://database.lichess.org/lichess_db_puzzle.csv.zst

--2025-01-03 15:20:07--  https://database.lichess.org/lichess_db_puzzle.csv.zst
Resolving database.lichess.org (database.lichess.org)... 141.95.66.62, 2001:41d0:700:5e3e::
Connecting to database.lichess.org (database.lichess.org)|141.95.66.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 227941696 (217M) [application/octet-stream]
Saving to: ‘lichess_db_puzzle.csv.zst’

lichess_db_puzzle.c 100%[===================>] 217.38M  22.5MB/s    in 10s     

2025-01-03 15:20:18 (21.0 MB/s) - ‘lichess_db_puzzle.csv.zst’ saved [227941696/227941696]



In [ ]:
!pip install zstandard

In [ ]:
import zstandard as zstd

# Specify the input and output file names
input_file = 'lichess_db_puzzle.csv.zst'
output_file = 'lichess_db_puzzle.csv'

# Open the input file and decompress it
with open(input_file, 'rb') as compressed:
    with open(output_file, 'wb') as decompressed:
        dctx = zstd.ZstdDecompressor()
        dctx.copy_stream(compressed, decompressed)

print('Decompression complete. The file is saved as', output_file)

Decompression complete. The file is saved as lichess_db_puzzle.csv


In [ ]:
!head -n 50 {output_file}

PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags
00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1902,76,95,7226,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,
0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - - 2 27,d3d6 f8d8 d6d8 f6d8,1512,74,96,29629,advantage endgame short,https://lichess.org/F8M8OS71#53,
0008Q,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1300,75,90,666,advantage endgame rookEndgame short,https://lichess.org/MQSyb3KW#127,
0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP1P/RN1QR1K1 b - - 1 16,b6c5 e2g4 h3g4 d1g4,1078,74,87,573,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Variation
000VW,r4r2/1p3pkp/p5p1/3R1N1Q/3P4/8/P1q2P2/3R2K1 b - - 3 25,g6f5 d5c5 c2e4 h5g5 g7h8 g5f6,2877,100,91,154,crushing endgame long,https://lichess.org/e9AY2m5j/black#50,
000Vc,8/8/4k1p1/2KpP2p/5PP1

In [ ]:
import os

# Check if the decompressed file exists
if os.path.exists(output_file):
    print(f"{output_file} has been successfully created.")
else:
    print("Decompression failed.")

lichess_db_puzzle.csv has been successfully created.


In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Read CSV") \
    .getOrCreate()

# Read the CSV file into a DataFrame
df = spark.read.csv(output_file, header=True, inferSchema=True)

# Get the first 50 rows
first_50_rows = df.limit(50)

# Show the results
first_50_rows.show()

# Stop the Spark session
spark.stop()

+--------+--------------------+--------------------+------+---------------+----------+-------+--------------------+--------------------+--------------------+
|PuzzleId|                 FEN|               Moves|Rating|RatingDeviation|Popularity|NbPlays|              Themes|             GameUrl|         OpeningTags|
+--------+--------------------+--------------------+------+---------------+----------+-------+--------------------+--------------------+--------------------+
|   00008|r6k/pp2r2p/4Rp1Q/...|f2g3 e6e7 b2b1 b3...|  1902|             76|        95|   7226|crushing hangingP...|https://lichess.o...|                NULL|
|   0000D|5rk1/1p3ppp/pq3b2...| d3d6 f8d8 d6d8 f6d8|  1512|             74|        96|  29629|advantage endgame...|https://lichess.o...|                NULL|
|   0008Q|8/4R3/1p2P3/p4r2/...| e7f7 f5e5 e2f1 e5e6|  1300|             75|        90|    666|advantage endgame...|https://lichess.o...|                NULL|
|   0009B|r2qr1k1/b1p2ppp/p...| b6c5 e2g4 h3g4 d1g4|

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Read CSV and Filter") \
    .getOrCreate()

# Read the CSV file
df = spark.read.csv(output_file, header=True, inferSchema=True)

# Show the first 50 rows of the DataFrame
df_first_50 = df.limit(500)

# Filter the DataFrame for the column 'Themes' that contains 'mate'
filtered_df = df_first_50.filter(col('Themes').contains('mate'))

# Collect and print the results
results = filtered_df.select('Themes').collect()
for row in results:
    print(row['Themes'])

# Stop the Spark session
spark.stop()

endgame mate mateIn2 short
mate mateIn2 middlegame short
backRankMate endgame mate mateIn2 short
bishopEndgame endgame mate mateIn1 oneMove
bodenMate hangingPiece mate mateIn1 middlegame oneMove
mate mateIn2 middlegame short
advancedPawn attraction mate mateIn2 middlegame promotion short
endgame long mate mateIn3
mate mateIn1 middlegame oneMove
endgame mate mateIn1 oneMove queensideAttack
endgame hangingPiece master mate mateIn1 oneMove
endgame master mate mateIn1 oneMove rookEndgame
kingsideAttack mate mateIn1 middlegame oneMove
endgame mate mateIn1 oneMove
backRankMate endgame fork mate mateIn2 short
attackingF2F7 mate mateIn1 middlegame oneMove
backRankMate mate mateIn2 middlegame short
endgame mate mateIn1 oneMove
mate mateIn1 middlegame oneMove
endgame mate mateIn2 short
kingsideAttack mate mateIn1 middlegame oneMove
kingsideAttack long mate mateIn3 middlegame sacrifice
endgame mate mateIn1 oneMove
endgame mate mateIn2 pin short
backRankMate endgame hangingPiece mate mateIn2 short

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Step 1: Initialize a Spark session
spark = SparkSession.builder \
    .appName("Filter CSV Example") \
    .getOrCreate()

# Step 2: Read the CSV file
df = spark.read.csv(output_file, header=True, inferSchema=True)

# Step 3: Filter rows where 'Themes' contains 'mate'
filtered_df = df.filter(col('Themes').contains('mate'))

# Step 4: Write the resulting DataFrame to a single CSV file

filtered_output_file = 'lichess_db_puzzle_mate.csv'  # Modify with your desired output path
# filtered_df.coalesce(1).write.csv(filtered_output_file, header=True, mode='overwrite')
filtered_df.toPandas().to_csv(filtered_output_file, header=True, index=False)

# Stop the Spark session
spark.stop()

In [ ]:
# remove outuput file or directory
# !rm -r {filtered_output_file}

In [ ]:
# Use the ! to run the wc command from within Jupyter notebook
lines_count = !wc -l < {filtered_output_file}
print(f"Found: {lines_count[0]} lines containing a final mate position.")

Found: 1230152 lines containing a final mate position.


In [ ]:
!head -n 50 {filtered_output_file}

PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags
000Zo,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w - - 0 35,e5f6 e8e1 g1f2 e1f1,1353,75,86,622,endgame mate mateIn2 short,https://lichess.org/n8Ff742v#69,
000hf,r1bqk2r/pp1nbNp1/2p1p2p/8/2BP4/1PN3P1/P3QP1P/3R1RK1 b kq - 0 19,e8f7 e2e6 f7f8 e6f7,1511,75,91,541,mate mateIn2 middlegame short,https://lichess.org/71ygsFeE/black#38,Horwitz_Defense Horwitz_Defense_Other_variations
001Wz,4r1k1/5ppp/r1p5/p1n1RP2/8/2P2N1P/2P3P1/3R2K1 b - - 0 21,e8e5 d1d8 e5e8 d8e8,1125,82,88,71,backRankMate endgame mate mateIn2 short,https://lichess.org/84RH3LaP/black#42,
001cr,8/3B2pp/p5k1/2p3P1/1p1p1K2/8/1P6/8 b - - 0 38,c5c4 d7e8,1792,76,91,991,bishopEndgame endgame mate mateIn1 oneMove,https://lichess.org/7FFNwibw/black#76,
001gi,r6r/1pNk1ppp/2np4/b3p3/4P1b1/N1Q5/P4PPP/R3KB1R w KQ - 3 18,c7a8 a5c3,824,120,62,239,bodenMate hangingPiece mate mateIn1 middlegame oneMove,https://lichess.org/1sV2Hr22#35,Sicilian_Defense Sicili

In [ ]:
!ls -lh

total 1.2G
-rw-r--r-- 1 root root 801M Jan  3 15:20 lichess_db_puzzle.csv
-rw-r--r-- 1 root root 218M Jan  2 06:56 lichess_db_puzzle.csv.zst
-rw-r--r-- 1 root root 210M Jan  3 15:23 lichess_db_puzzle_mate.csv
drwxr-xr-x 1 root root 4.0K Dec 19 14:20 sample_data


In [ ]:
# remove initial compressed version of lichess_db and uncompressed version as well
!rm lichess_db_puzzle.csv.zst
!rm lichess_db_puzzle.csv

In [ ]:
import pandas as pd

filtered_output_file = 'lichess_db_puzzle_mate.csv'

# Load CSV file into a DataFrame
df = pd.read_csv(filtered_output_file)

# Process each row using itertuples
line=0
for row in df.itertuples(index=True):
    # Custom processing logic
    print(f"Row {line} {df['PuzzleId'][line]} | {df['FEN'][line]} | {df['Moves'][line]}|")

    line = line + 1
    if line > 10:
      break



Row 0 000Zo | 4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w - - 0 35 | e5f6 e8e1 g1f2 e1f1|
Row 1 000hf | r1bqk2r/pp1nbNp1/2p1p2p/8/2BP4/1PN3P1/P3QP1P/3R1RK1 b kq - 0 19 | e8f7 e2e6 f7f8 e6f7|
Row 2 001Wz | 4r1k1/5ppp/r1p5/p1n1RP2/8/2P2N1P/2P3P1/3R2K1 b - - 0 21 | e8e5 d1d8 e5e8 d8e8|
Row 3 001cr | 8/3B2pp/p5k1/2p3P1/1p1p1K2/8/1P6/8 b - - 0 38 | c5c4 d7e8|
Row 4 001gi | r6r/1pNk1ppp/2np4/b3p3/4P1b1/N1Q5/P4PPP/R3KB1R w KQ - 3 18 | c7a8 a5c3|
Row 5 001om | 5r1k/pp4pp/5p2/1BbQp1r1/6K1/7P/1PP3P1/3R3R w - - 2 26 | g4h4 c5f2 g2g3 f2g3|
Row 6 001w5 | 1rb2rk1/q5P1/4p2p/3p3p/3P1P2/2P5/2QK3P/3R2R1 b - - 0 29 | f8f7 c2h7 g8h7 g7g8q|
Row 7 001wR | 6nr/pp3p1p/k1p5/8/1QN5/2P1P3/4KPqP/8 b - - 5 26 | b7b5 b4a5 a6b7 c4d6 b7b8 a5d8|
Row 8 001wb | r3k2r/pb1p1ppp/1b4q1/1Q2P3/8/2NP1Pn1/PP4PP/R1B2R1K w kq - 1 17 | h2g3 g6h5|
Row 9 002CP | r5k1/pp4pp/4p1q1/4p3/3n4/P5P1/1PP2Q1P/2KR1R2 w - - 4 24 | f2e3 g6c2|
Row 10 002Z9 | 4r1k1/1p2R1p1/p2p2Pp/P1pP4/5q2/1R3p2/1P1Q3P/5B1K b - - 0 34 | f4d2 e7e8|


In [ ]:
pip install python-chess

In [ ]:
import pandas as pd
import chess
import numpy as np
from datetime import datetime, timedelta

def getPiecePositions(piece_type, piece_color):
    # Now let's find all the pieces of given type and their positions
    positions = []

    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None and piece.piece_type == piece_type and piece.color == piece_color:
          # In python-chess, files are numbered from 0 to 7 (where 0 corresponds to the 'a' file and 7 corresponds to the 'h' file),
          # and ranks also range from 0 to 7 (with 0 being rank 1 and 7 being rank 8).
          # You can adjust these values if you need them to match standard chess notation.
          x = int(chess.square_file(square))
          y = int(chess.square_rank(square))
          positions.append({"coord":[x, y], "square":chess.square_name(square)})  # Store the position in algebraic notation

    return positions

filtered_output_file = 'lichess_db_puzzle_mate.csv'

# Load CSV file into a DataFrame
df = pd.read_csv(filtered_output_file)
df['Rook_Knight'] = False
df['puzzle_ref'] = 'https://lichess.org/training/' + df['PuzzleId']

# Process each row using itertuples
line=0
matched_lines=0
start_time = datetime.now()
for row in df.itertuples(index=True):

    if (line % 100000) == 0:
      # Calculate the time delta
      time_delta = datetime.now() - start_time
      print(f"{time_delta}: puzzles searched: {line} matches: {matched_lines}")

    #if df['PuzzleId'][line] != 'pSskg':
    #  line = line + 1
    #  continue

    moves_to_apply = df['Moves'][line].split()

    # Print the board position
    #print(board)
    #display(board)
    #print('Making moves')
    # Load the position from the FEN string
    board = chess.Board(df['FEN'][line])

    # Apply each move to the board
    for move in moves_to_apply:
        # Create a Move object from the UCI string
        chess_move = chess.Move.from_uci(move)

        # Check if the move is legal before making it
        if chess_move in board.legal_moves:
            board.push(chess_move)
        else:
            print(f"Illegal move: {move}")

    mated_color = None
    mating_color = None
    # Check if the position is checkmate
    if board.is_checkmate():
        # Determine the color of the king being mated
        # If it's white's turn to move, then black's king is in checkmate; otherwise, white's king is in checkmate.
        if board.turn == chess.WHITE:
            mated_color = chess.WHITE
            mating_color = chess.BLACK
        else:
            mated_color = chess.BLACK
            mating_color = chess.WHITE
    else:
        line = line + 1
        continue

    # find pieces that deliver the check
    # Determine the square of the king in check

    # Now let's find all the white queens and their positions
    king = getPiecePositions(chess.KING, mated_color)
    knights = getPiecePositions(chess.KNIGHT, mating_color)
    rooks = getPiecePositions(chess.ROOK, mating_color)

    # check if there is a queen and rook available in this position
    if len(knights) == 0 or len(rooks) == 0:
      line = line + 1
      continue

    king_pos = np.array(king[0]["coord"])

    valid_rooks = []           # Find a valid distance between a king and rook
    for rook in rooks:
      rook_pos = np.array(rook["coord"])
      delta = king_pos - rook_pos
      # Check if rook is in right or left square of king
      if abs(delta[1])  == 0 and abs(delta[0]) == 1:
        valid_rooks.append(rook)

      # Check if rook is above or below square of king
      if abs(delta[0])  == 0 and abs(delta[1]) == 1:
        valid_rooks.append(rook)

    if len(valid_rooks) == 0:
      line = line + 1
      continue

    valid_knights = []           # Find a valid distance between a king and queen
    for knight in knights:
      knight_pos = np.array(knight["coord"])
      delta = king_pos - knight_pos
      if abs(delta[0])  == 0 and abs(delta[1]) == 2 or abs(delta[0])  == 2 and abs(delta[1]) == 0:
        valid_knights.append(knight)

    if len(valid_knights) == 0:
      line = line + 1
      continue

    mating_rooks = []       # validate rook and knight positions as file or rank
    mating_knights = []
    for knight in valid_knights:
      for rooks in valid_rooks:
        rook_pos = np.array(rook["coord"])
        knight_pos = np.array(knight["coord"])
        delta = rook_pos - knight_pos

        # Knight and rook must be on the same rank or file next to each other
        # and should form a triangle with the king
        # 1) Queen and Rook are on one file (same x coordinate)
        if abs(delta[0]) == 0 and abs(delta[1]) == 1:
          mating_rooks.append(rook)
          mating_knights.append(knight)
        else:
          # 2) Queen and Rook are on one rank (same y coordinate)
          if abs(delta[1]) == 0 and abs(delta[0]) == 1:
            mating_rooks.append(rook)
            mating_knights.append(knight)

    if len(mating_rooks) == 0 or len(mating_knights) == 0:
      line = line + 1
      continue

    # Print the board position
    # print(board)
    #print(f"Row {line} {df['PuzzleId'][line]} | {df['FEN'][line]} | {moves_to_apply}| | {df['GameUrl'][line]}")
    #print(f"Positions of {'white' if mated_color else 'black'}'s king {king} {'white' if mating_color else 'black'} queens {queens} and rooks {rooks}")
    #print(f"{'white' if mated_color else 'black'}'s king is checkmated by {'white' if mating_color else 'black'}.")
    #print(f"Positions of {'white' if mated_color else 'black'}'s king {king} mated by {'white' if mating_color else 'black'} queens {mating_queens} and rooks {mating_rooks}")
    #print(f"Mated by {'white' if mating_color else 'black'} queens {mating_queens} and rooks {mating_rooks} -> center: {mating_deltas}")
    #display(board)
    #print('------------------------------------------------------')
    #print()

    df.loc[line, "Rook_Knight"] = True

    # Print the board position #################################################
    #print(board)
    #display(board)
    #print('DEBUGGING')

    line = line + 1
    matched_lines = matched_lines + 1

    # Break after x matches (for debugging only - when developing the script)
    #if (matched_lines > 100):
    #  break


0:00:00.002954: puzzles searched: 0 matches: 0
0:00:47.284819: puzzles searched: 100000 matches: 150
0:01:37.280633: puzzles searched: 200000 matches: 293
0:02:26.845910: puzzles searched: 300000 matches: 422
0:03:13.871791: puzzles searched: 400000 matches: 577
0:03:59.366078: puzzles searched: 500000 matches: 706
0:04:44.644627: puzzles searched: 600000 matches: 844
0:05:32.386120: puzzles searched: 700000 matches: 999
0:06:18.312096: puzzles searched: 800000 matches: 1137
0:07:04.557416: puzzles searched: 900000 matches: 1296
0:07:49.911761: puzzles searched: 1000000 matches: 1433
0:08:38.158952: puzzles searched: 1100000 matches: 1589
0:09:25.325976: puzzles searched: 1200000 matches: 1759


In [ ]:
print(f"{matched_lines} Vucovic patterns found.")


# Output the first 100 rows of the DataFrame
filtered_df = df[df['Rook_Knight'] == True] # Filter where 'Rook_Knight' is True
result = filtered_df.head(100)  # Get the first 100 rows

print(result)  # Display the result

1809 Vucovic patterns found.
      PuzzleId                                                FEN  \
180      00VIe               8/8/8/P6p/8/2RnkN2/r7/3K4 w - - 1 60   
336      00yzf   3rk3/3N2R1/4P1pr/3pP3/2p5/6p1/PP6/2K5 b - - 3 29   
2136     06MoJ          6R1/1P6/8/7p/4P2P/4kn2/1r6/5K2 w - - 3 39   
2242     06l4B     1k6/1p6/1Qn5/PN1p4/5b2/7P/1Pr1KP2/8 w - - 0 34   
2648     07uNx      3Q4/5pkp/4P1p1/8/7P/6P1/rr3n2/5RK1 w - - 0 34   
...        ...                                                ...   
72723    3ePoO            8/8/5RK1/5n2/B4k1p/1P6/6r1/8 w - - 3 61   
73059    3fRfd  6k1/6pp/p7/P2P4/1P2N3/3np1P1/3r1r2/R2K3R w - -...   
74792    3kkF2  3r2r1/Npk2p1p/B2pb3/2q5/3QP3/7P/P1P1KbP1/1R5R ...   
75399    3mp9w  6k1/4q1pp/4P3/3p1p2/p2Nn2Q/P1P4P/1r3rP1/R1R4K ...   
75486    3n7Ru  2r2b1r/3np3/pq2Qkpn/1p4N1/2p2P2/2P5/PPB3PP/4RR...   

                     Moves  Rating  RatingDeviation  Popularity  NbPlays  \
180              f3e1 a2d2    1974               79          94   

In [ ]:
# Assuming 'df' is your DataFrame
filtered_df.to_csv('lichess_db_puzzle_mate_vucovic.csv', index=False, header=True)

In [ ]:
!ls -lh

total 211M
-rw-r--r-- 1 root root 210M Jan  3 12:43 lichess_db_puzzle_mate.csv
-rw-r--r-- 1 root root 1.2M Jan  3 13:37 lichess_db_puzzle_mate_triangle.csv
drwxr-xr-x 1 root root 4.0K Dec 19 14:20 sample_data


In [ ]:
# prompt: store the resulting CSV file in google drive

!ls -lh
from google.colab import drive

drive.mount('/content/drive')

# Specify the file path in your Google Drive
drive_file_path = '/content/drive/MyDrive/lichess_db_puzzle_mate_vucovic.csv'

# Copy the file to your Google Drive
!cp lichess_db_puzzle_mate_vucovic.csv {drive_file_path}

total 211M
drwx------ 6 root root 4.0K Jan  3 16:19 drive
-rw-r--r-- 1 root root 210M Jan  3 15:23 lichess_db_puzzle_mate.csv
-rw-r--r-- 1 root root 356K Jan  3 16:19 lichess_db_puzzle_mate_vucovic.csv
drwxr-xr-x 1 root root 4.0K Dec 19 14:20 sample_data
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
